# QuantumSafeNet-RAG

In [ ]:
!pip install PyPDF2
!pip install transformers torch
!pip install sentence_transformers
!pip install sentencepiece
!pip install langchain_openai 
!pip install openai 
!pip install qdrant-client

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing 

# SentenceTransformer Embedding

In [5]:
from sentence_transformers import SentenceTransformer
def get_sentenceTF_embeddings(sentences):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  embeddings =[]
  for chunk in sentences:
    embeddings.append(model.encode(chunk))
  print(len(embeddings))
  return embeddings

def Embed_stenteceTF(sentence):
  model = SentenceTransformer('all-MiniLM-L6-v2')
  return model.encode(sentence)

2025-04-19 04:33:53.630512: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745037233.836672      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745037233.899017      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# **Sementic Chuncking**

In [6]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.cluster import KMeans


def get_bert_embeddings(sentences):
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Tokenize sentences
    inputs = tokenizer(sentences, return_tensors='pt', padding=True, truncation=True, max_length=128)

    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Use the [CLS] token representation as the sentence embedding
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()

    return embeddings

def semantic_chunking_with_attention(text, n_clusters=5, max_chunk_size=512):
    # Split the input text into sentences based on '.'
    sentences = [sentence.strip() for sentence in text.split('.') if sentence.strip()]

    # Get BERT embeddings for sentences
    embeddings = get_bert_embeddings(sentences)

    # Perform KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=5).fit(embeddings)
    labels = kmeans.labels_

    # Group sentences by cluster
    clusters = {}
    for sentence, label in zip(sentences, labels):
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(sentence)

    # Create chunks from clusters with max_chunk_size constraint
    chunks = []
    for cluster in clusters.values():
        current_chunk = ""
        for sentence in cluster:
            if len(current_chunk) + len(sentence) + 1 <= max_chunk_size:  # +1 for the period
                current_chunk += sentence + ". "
            else:
                chunks.append(current_chunk.strip())
                current_chunk = sentence + ". "
        if current_chunk:
            chunks.append(current_chunk.strip())

    return chunks

# **Recursive Chunking**

In [7]:
def division_Chunk(text, chunk_size=400):
    sentences = text.split(".")
    chunk1 = ""
    chunks = []

    for sentence in sentences:
        word_count = len(sentence.split())

        if word_count > chunk_size:
            chunks.extend(Recursive_Chunking(sentence, chunk_size))
        else:
            if len(chunk1.split()) + word_count <= chunk_size:
                chunk1 += sentence + "\n"
            else:
                chunks.append(chunk1.strip())
                chunk1 = sentence + "\n"

    if chunk1.strip():
        chunks.append(chunk1.strip())

    return chunks

def Recursive_Chunking(text, chunk_size=1000):
    chunks = text.split("\n\n")
    #print(len(chunks))
    result_chunks = []

    for chunk in chunks:
        word_count = len(chunk.split())

        if word_count > chunk_size:
            middle = int(0.8 * word_count)
            words = chunk.split()
            part1 = " ".join(words[:middle])
            part2 = " ".join(words[middle:])

            result_chunks.extend(division_Chunk(part1, chunk_size))
            result_chunks.extend(division_Chunk(part2, chunk_size))
        else:
            result_chunks.append(chunk.strip())

    return result_chunks

# Add_Data_From_PDF_file

In [8]:
from PyPDF2 import PdfReader
import re

def read_pdf_file(file_path):
    points = []
    # Open the PDF file
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        # Iterate through each page in the PDF
        for page in pdf_reader.pages:
            text = page.extract_text()
            if text:
                points.append(text.strip())
    return points

def replace_double_spaces(text):
    # Replace all occurrences of non-breaking spaces and zero-width spaces
    cleaned_text = re.sub(r'\xa0', ' ', text)
    cleaned_text1 = re.sub(r'\u200b', '', cleaned_text)
    return cleaned_text1

def process_pdf_files(file_paths, chunking_type, max_chunk_size, number_of_clusters=20):
    all_embeddings = []
    all_chunks = []

    for file_path in file_paths:
        print(f"Processing: {file_path}")
        Pneumonia = read_pdf_file(file_path)
        
        # Clean the text
        for i in range(len(Pneumonia)):
            Pneumonia[i] = replace_double_spaces(Pneumonia[i])
        
        Pneumonia_Str = "\n".join(Pneumonia)
        
        # Chunking
        if chunking_type == "Recursive":
            chunks = Recursive_Chunking(Pneumonia_Str, chunk_size=max_chunk_size)
        elif chunking_type == "Semantic":
            chunks = semantic_chunking_with_attention(Pneumonia_Str, n_clusters=number_of_clusters, max_chunk_size=max_chunk_size)
        
        # Get embeddings
        embedding = get_sentenceTF_embeddings(chunks)
        
        all_embeddings.extend(embedding)
        all_chunks.extend(chunks)
    
    return all_embeddings, all_chunks

In [6]:
# Provide the list of PDF file paths
file_paths = [
    '/kaggle/input/quantumsafenet/CS-453 Final Year Project-I Report.pdf'
]

# Process all the files and obtain embeddings and chunks
embeddings, chunks = process_pdf_files(file_paths, chunking_type='Semantic', max_chunk_size=600, number_of_clusters=20)

# Print the lengths of the chunks and embeddings
print(len(chunks))
print(len(embeddings))

Processing: /kaggle/input/quantumsafenet/CS-453 Final Year Project-I Report.pdf


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

245
245
245


# **PrepareData**

In [7]:
from qdrant_client.models import PointStruct
import uuid

# Create PointStruct objects
points = [
    PointStruct(
        id=str(uuid.uuid4()),  # Use a unique ID for each document
        vector=embedding,  # Ensure this is a list of floats
        payload={"text": doc}  # Optional metadata
    )
    for i, (doc, embedding) in enumerate(zip(chunks, embeddings))
]

In [ ]:
from qdrant_client import QdrantClient

qdrant_client = QdrantClient(
    url="YOUR_QDRANT_API_URL", 
    api_key="YOUR_QDRANT_API",
)

print(qdrant_client.get_collections())

collections=[CollectionDescription(name='quantumsafenetbot')]


In [9]:
from qdrant_client.models import CollectionParams, VectorParams

# Define the dimension of your vectors and distance metric
vector_dim = 384  # Dimension of your vectors
distance_metric = "Cosine"  # Distance metric (use "Cosine" as string)

# Define vector configuration
vector_params = VectorParams(size=vector_dim, distance=distance_metric)

# Create the collection
qdrant_client.create_collection(
    collection_name="quantumsafenetbot",
    vectors_config=vector_params  # Correct argument name
)

True

In [10]:
# Insert points into Qdrant
qdrant_client.upsert(
    collection_name="quantumsafenetbot",
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
# llm_model.generate(concatenated_text,max_tokens=2048,max_new_tokens=2048 )

In [9]:
query = ["What does this QuantumSafeNet system is about?"]
query_embedding_response = get_sentenceTF_embeddings(query)
# print(query_embedding_response)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

1


In [10]:
# Assuming query_embedding_response is a list of lists or a NumPy array
query_embedding = query_embedding_response[0].tolist()  # Convert to list if necessary

In [13]:
# Perform similarity search
results = qdrant_client.search(
    collection_name="quantumsafenetbot",
    query_vector=query_embedding,
)
# Print results
for result in results:
    print(result)

/tmp/ipykernel_31/4107776642.py:2: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


id='d2e15b3b-ca1d-435f-adc4-d87ecc6277eb' version=0 score=0.6087171 payload={'text': '1 Post-Quantum Cryptography Integration\n•Objective:\nTo enhance the security of QuantumSafeNet by integrating post-quantum crypto-\ngraphic algorithms.'} vector=None shard_key=None order_value=None
id='1fa5ae01-1e0d-4d61-9472-c907be40a492' version=0 score=0.57549894 payload={'text': 'QuantumSafeNet combines\npost-quantum encryption for future-proof security with AI-powered threat detection to\nmonitor and respond to risks in real time—all without slowing you down or making things\nharder to use. With QuantumSafeNet, organizations can work confidently, knowing their data is secure\nnow and in the future. 2 Project Objectives\nOur project aimed to:\n•Build a secure and scalable network that grows with an organization’s needs while\nkeeping data safe.'} vector=None shard_key=None order_value=None
id='b111078c-24fa-4712-9527-e47e397a962d' version=0 score=0.54836905 payload={'text': '3 Purpose\nThe purpos

In [ ]:
# NVIDIA NIM API
from openai import OpenAI

client = OpenAI(
  base_url = "https://integrate.api.nvidia.com/v1",
  api_key = "YOUR_NVIDIA_API_KEY"
)

In [18]:
def custom_prompt(query: str, results):
    # Extract the page content from the results
    source_knowledge = "\n".join([x.payload['text'] for x in results])
    
    # Create the augmented prompt
    augment_prompt = f"""Using the contexts below, answer the query in short paragraph:

    Contexts:
    {source_knowledge}

    Query: {query}"""
    
    return augment_prompt

In [ ]:
# print(custom_prompt(query, results))

In [19]:
messages = []
# messages.append({"role":"assistant", "content": full_response})
prompt = {"role":"system", "content": custom_prompt(query, results)}
messages.append(prompt)

res = client.chat.completions.create(
  model="meta/llama-3.1-8b-instruct",
  messages=messages,
  temperature=0.2,
  top_p=0.7,
  max_tokens=1024,
  stream=True
)

full_response = ""
for chunk in res:
  if chunk.choices[0].delta.content is not None:
#     print(chunk.choices[0].delta.content, end="")
    full_response += chunk.choices[0].delta.content
print(full_response)

QuantumSafeNet is a secure system designed to provide post-quantum cryptography and AI-powered threat detection for remote workforce access to company resources. It aims to balance robust security measures with practical usability, making it suitable for real-world enterprise environments. The system is built to handle current and future security threats, including those posed by quantum computing, while ensuring ease of use and minimizing the need to choose between security and usability.
